# Training GPT-2 with English Poetry

#### Don't run this exept if you have a gpu, torch and conda all installed

This notebook will train the model you give him, on the data given and there is a part where we can generate poems from the newly trained model.

You can put the files you want in the training and test data.

The instructions for training are based on those found at https://towardsdatascience.com/how-to-fine-tune-gpt-2-for-text-generation-ae2ea53bc272 or at https://colab.research.google.com/github/philschmid/fine-tune-GPT-2/blob/master/Fine_tune_a_non_English_GPT_2_Model_with_Huggingface.ipynb#scrollTo=V36gOIOfLHvB.  In fact, the model is pre-trained, and then **fine-tuned** to poetry data.  The training objective is, like for all language models, to predict the next token when given large amounts of training texts.

In [1]:
import torch
from tqdm import tqdm

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
torch.cuda.empty_cache()
torch.cuda.memory_summary()

True
1


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [19]:
f = open("data_english/english_poems_processed_train.txt", mode="r", encoding="ansi")
train = f.read()
f = open("data_english/english_poems_processed_test.txt", mode="r", encoding="utf-8")
test = f.read() # this is a smaller excerpt (4.5 MB) used for validation (during training)

print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 2210851
Test dataset length: 35245


In [4]:
from transformers import AutoTokenizer

In [20]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

train_path = 'data_english/english_poems_processed_train.txt'
test_path = 'data_english/english_poems_processed_test.txt'

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\etien/.cache\huggingface\hub\models--gpt2\snapshots\e7da7f221d5bf496a48136c0cd264e630fe9fcc8\config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
  

In [6]:
from transformers import TextDataset, DataCollatorForLanguageModeling

In [7]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

C:\Users\etien\anaconda3\envs\crpo\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

In [9]:
model = AutoModelWithLMHead.from_pretrained("gpt2-poetry-model-with-start-token")

C:\Users\etien\anaconda3\envs\crpo\lib\site-packages\transformers\models\auto\modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [10]:
training_args = TrainingArguments(
    output_dir="./gpt2-poetry-model-with-start-token", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    eval_steps = 500, # Number of update steps between two evaluations.
    save_steps=500, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps"
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [11]:
trainer.train()

C:\Users\etien\anaconda3\envs\crpo\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4722
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2960
  Number of trainable parameters = 124439808


Step,Training Loss,Validation Loss
500,2.024900,5.471236
1000,1.712200,5.620810
1500,1.738300,5.681308
2000,1.774800,5.539335
2500,1.870100,5.505610


***** Running Evaluation *****
  Num examples = 73
  Batch size = 32
Saving model checkpoint to ./gpt2-poetry-model-with-start-token\checkpoint-500
Configuration saved in ./gpt2-poetry-model-with-start-token\checkpoint-500\config.json
Model weights saved in ./gpt2-poetry-model-with-start-token\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 73
  Batch size = 32
Saving model checkpoint to ./gpt2-poetry-model-with-start-token\checkpoint-1000
Configuration saved in ./gpt2-poetry-model-with-start-token\checkpoint-1000\config.json
Model weights saved in ./gpt2-poetry-model-with-start-token\checkpoint-1000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 73
  Batch size = 32
Saving model checkpoint to ./gpt2-poetry-model-with-start-token\checkpoint-1500
Configuration saved in ./gpt2-poetry-model-with-start-token\checkpoint-1500\config.json
Model weights saved in ./gpt2-poetry-model-with-start-token\checkpoint-1500\pytorch_model.bin
***** Runn

TrainOutput(global_step=2960, training_loss=1.8501393911000845, metrics={'train_runtime': 1848.2822, 'train_samples_per_second': 25.548, 'train_steps_per_second': 1.601, 'total_flos': 3084552437760000.0, 'train_loss': 1.8501393911000845, 'epoch': 10.0})

In [12]:
trainer.save_model()

Saving model checkpoint to ./gpt2-poetry-model-with-start-token
Configuration saved in ./gpt2-poetry-model-with-start-token\config.json
Model weights saved in ./gpt2-poetry-model-with-start-token\pytorch_model.bin


In [13]:
torch.device("cuda")

device(type='cuda')

In [14]:
from transformers import pipeline
poetry = pipeline('text-generation', model='./gpt2-poetry-model-with-start-token', tokenizer='gpt2', device = 0)

loading configuration file ./gpt2-poetry-model-with-start-token\config.json
Model config GPT2Config {
  "_name_or_path": "./gpt2-poetry-model-with-start-token",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use

In [15]:
print(poetry('<start>Though as the weather goes, my mind would be all in trouble for me', max_length = 19)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start>Though as the weather goes, my mind would be all in trouble for me



In [16]:
print(len(poetry('<start>The snow is white and the sky is blue.\n<start>', max_length = 210)[0]['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


824


In [17]:
print(poetry('Do not stand by my grave and weep', max_length = 300)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Do not stand by my grave and weep

For pity only show thy woe
And love and pity may me lose

And she that did my sighs inspire
May pray for me in this wise

Thou most imperfect spirit, say
What is the cause of this almighty pain

Then say, and I will think it well
This eye which kindled this fire

And since that grief and sorrow I know
Does with it feed and combat fire

Thus may all pity prove deceiving
Thus doth love and this withen die

And then 't is but by hoping too much
May I still live on in love's despite

Thou most unjust and cruel god, to me unjust

Yet dost love but to torment me with a smile
Painful and certain doom

Lo, cruel god, my soul doth wish a thing unjust

Which never shall be done unjust to thee

Yet cruel god, I do requite thy wrong

O cruel god, by hard compulsion crave

O cruel god, by hard compulsion crave

O cruel god, since all my wrongs are vain

Therefore, since she in time hath been unjust

Can I for ever unjust be damned


Since thou hast cursed am I and

In [18]:
import random
import time

for i in tqdm(range(50)):
    
    # AABB
    samples = ['Do not stand by my grave and weep\nThe appeal for darkness is to steep\n\n', 'She walks in beauty, like the night\nI would love to be her knight\n\n', 'Life, believe, is not a dream\nYou can not wake up just with a scream\n\n', 'life is a tall tender tree\nSavor it like a cookie\n\n', 'The taste of marmelade is better with you\nYou light up my day when you lace your shoe\n\n','They were mad and in love\nShe was a wolf and he was a dove\n\n', 'My heart aches just thinking of you\nBut now I am left feeling blue\n\n', 'They were lonely and tired\nThat is when the shot was fired\n\n', 'I am writting this poem for you\nTo let you know that I will always be true\n\n', 'I long for the warmth of your smile\nand the sight of your teeth like a crocodile\n\n']
    
    # ABAB
    # samples = ['Do not stand by my grave and weep\n My memory of you remains\nThe appeal for darkness is to steep\nMy link with death feel like chains\n\n', 'She walks in beauty, like the night\n In a white dress like the moon\nI would love to be her knight\n On her path flowers bloom\n\n', 'Life, believe, is not a dream\nEven in the middle of a nightmare\nYou can not wake up just with a scream\nBut you still feel the scare\n\n','life is a tall tender tree\nLive it with all your soul\nSavor it like a cookie\nBut beware not to fall\n\n']
              
    
    first = random.sample(samples, 1)[0]
    
    begin = time.time()
    
    poem = poetry(first, max_length = 300)[0]['generated_text']
    
    # write on file
    
    with open('poems_generated_english/TEST_POEM.txt', 'a', encoding='utf-8') as f:
        f.write(poem)

    end = time.time()
    
    print(f"Total runtime of the program is {end - begin}")

  2%|█▋                                                                                 | 1/50 [00:04<03:23,  4.14s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Total runtime of the program is 4.143949031829834


  4%|███▎                                                                               | 2/50 [00:08<03:18,  4.13s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Total runtime of the program is 4.126929044723511


  6%|████▉                                                                              | 3/50 [00:12<03:12,  4.09s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Total runtime of the program is 4.36732292175293


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [03:15<00:03,  4.00s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Total runtime of the program is 3.84368896484375


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [03:19<00:00,  3.98s/it]

Total runtime of the program is 3.79288649559021
